In [17]:
# !wget https://files.slack.com/files-pri/T7ZHK7DPG-F7Y67UV6D/download/train_sms.csv
!wget https://files.slack.com/files-pri/T7ZHK7DPG-F7Y67UV6D/download/train_sms.csv

--2017-11-12 06:29:58--  https://files.slack.com/files-pri/T7ZHK7DPG-F7Y67UV6D/download/train_sms.csv
Resolving files.slack.com (files.slack.com)... 13.32.173.5
Connecting to files.slack.com (files.slack.com)|13.32.173.5|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://wxplusb.slack.com/?redir=%2Ffiles-pri%2FT7ZHK7DPG-F7Y67UV6D%2Fdownload%2Ftrain_sms.csv [following]
--2017-11-12 06:29:58--  https://wxplusb.slack.com/?redir=%2Ffiles-pri%2FT7ZHK7DPG-F7Y67UV6D%2Fdownload%2Ftrain_sms.csv
Resolving wxplusb.slack.com (wxplusb.slack.com)... 13.32.170.15
Connecting to wxplusb.slack.com (wxplusb.slack.com)|13.32.170.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘train_sms.csv.1’

train_sms.csv.1         [ <=>                ]  68.62K  --.-KB/s    in 0.06s   

2017-11-12 06:29:58 (1.03 MB/s) - ‘train_sms.csv.1’ saved [70271]



In [2]:
data_train = pd.read_csv('TRAIN_SMS.csv', encoding='cp1252')

NameError: name 'pd' is not defined

In [1]:
_map = {'ham': 0, 'info': 1, 'spam': 2}
# data_train['Label'] = data_train['Label'].map(_map)
# data_train.head(3)

In [ ]:
# author - Richard Liao
# Dec 26 2016
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

data_train = pd.read_csv('TRAIN_SMS.csv', encoding='cp1252')
data_train['Label'] = data_train['Label'].map(_map)

print(data_train.head())

texts = []
labels = []
print (data_train.head())
for idx in range(data_train.Message.shape[0]):
    text = BeautifulSoup(data_train.Message[idx])
    texts.append(clean_str(text.get_text()))
    labels.append(data_train.Label[idx])
    

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print (y_val.sum(axis=0))

GLOVE_DIR = ""
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'sms_model.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(3, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - simplified convolutional neural network")
model.summary()
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=5, batch_size=128, callbacks=callbacks_list)

Using Theano backend.


   Label                                            Message
0      0                 oh how abt 2 days before Christmas
1      1  Welcome to OVATION HOLD R.No. 184, 114, 395, 3...
2      1  Thank you for using your ICICI bank CREDITcard...
3      0  schedule a meeting with the entire team in the...
4      0                                Tommy is my brother
   Label                                            Message
0      0                 oh how abt 2 days before Christmas
1      1  Welcome to OVATION HOLD R.No. 184, 114, 395, 3...
2      1  Thank you for using your ICICI bank CREDITcard...
3      0  schedule a meeting with the entire team in the...
4      0                                Tommy is my brother


/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /anaconda/envs/py35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "https://itunes.apple.com/us/album/adventure-of-a-lifetime/id7266550223?i=PHONENUMBER" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that docume

Found 39161 unique tokens.
Shape of data tensor: (30000, 1000)
Shape of label tensor: (30000, 3)
Number of positive and negative reviews in traing and validation set 
[  8044.  10721.   5235.]
[ 1956.  2654.  1390.]
Total 2273 word vectors in Glove 6B 100d.
model fitting - simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         3916200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)         

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:132: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 24000 samples, validate on 6000 samples
Epoch 1/5
24000/24000 [==============================] - 369s - loss: 0.7571 - acc: 0.7530 - val_loss: 0.1854 - val_acc: 0.9307
Epoch 2/5
23936/24000 [============================>.] - ETA: 0s - loss: 1.0496 - acc: 0.8298  

In [ ]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,

                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=20, batch_size=50)

# model.save_weights('sms_graph.h5')

Using Theano backend.


   Label                                            Message
0      0                 oh how abt 2 days before Christmas
1      1  Welcome to OVATION HOLD R.No. 184, 114, 395, 3...
2      1  Thank you for using your ICICI bank CREDITcard...
3      0  schedule a meeting with the entire team in the...
4      0                                Tommy is my brother
   Label                                            Message
0      0                 oh how abt 2 days before Christmas
1      1  Welcome to OVATION HOLD R.No. 184, 114, 395, 3...
2      1  Thank you for using your ICICI bank CREDITcard...
3      0  schedule a meeting with the entire team in the...
4      0                                Tommy is my brother


/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /anaconda/envs/py35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "https://itunes.apple.com/us/album/adventure-of-a-lifetime/id7266550223?i=PHONENUMBER" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that docume

Found 39161 unique tokens.
Shape of data tensor: (30000, 1000)
Shape of label tensor: (30000, 3)
Number of positive and negative reviews in traing and validation set 
[  8060.  10625.   5315.]
[ 1940.  2750.  1310.]
Total 2273 word vectors in Glove 6B 100d.
model fitting - simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         3916200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)         

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:128: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 24000 samples, validate on 6000 samples
Epoch 1/10
24000/24000 [==============================] - 369s - loss: 0.3277 - acc: 0.8763 - val_loss: 0.0291 - val_acc: 0.9915

In [ ]:
model.save_weights('sms_graph.h5')

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)